# Optiver Realized Volatility Prediction

## Apply your data science skills to make financial markets better


<a href= "https://www.optiver.com/insights/guides/options-volatility/">Volatility</a> is one of the most prominent terms you’ll hear on any trading floor – and for good reason. In financial markets, volatility captures the amount of fluctuation in prices. High volatility is associated to periods of market turbulence and to large price swings, while low volatility describes more calm and quiet markets. For trading firms like Optiver, accurately predicting volatility is essential for the trading of options, whose price is <a href= "https://www.optiver.com/insights/guides/options-pricing/">directly related to the volatility</a> of the underlying product.

## Understanding the data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn


In [3]:
dftt = pd.read_csv("train.csv")
print(dftt.info())
dftt.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428932 entries, 0 to 428931
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   stock_id  428932 non-null  int64  
 1   time_id   428932 non-null  int64  
 2   target    428932 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 9.8 MB
None


,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [10]:
id_count = pd.DataFrame(dftt.stock_id.value_counts()).sort_index()
print(id_count)

     stock_id
0        3830
1        3830
2        3830
3        3830
4        3830
..        ...
122      3830
123      3830
124      3830
125      3830
126      3830

[112 rows x 1 columns]


In [11]:
dftt.tail()

,stock_id,time_id,target
428927,126,32751,0.003461
428928,126,32753,0.003113
428929,126,32758,0.004070
428930,126,32763,0.003357
428931,126,32767,0.002090


In [12]:
dfs = pd.read_csv("sample_submission.csv")
print(dfs.info())
dfs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   row_id  3 non-null      object 
 1   target  3 non-null      float64
dtypes: float64(1), object(1)
memory usage: 176.0+ bytes
None


,row_id,target
0,0-4,0.003048
1,0-32,0.003048
2,0-34,0.003048


## Selecting data

## Working on the models